![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [10]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("kliy2ZLKP46T-EtH4mHlU7bX7twpU4LCDGDKaTHYFdgV ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [11]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [12]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 900,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [13]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [14]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


kliy2ZLKP46T-EtH4mHlU7bX7twpU4LCDGDKaTHYFdgV ········


In [15]:
!pip install "pydantic<2.0"
!pip install --upgrade fastapi
!pip install  uvicorn pyngrok nest-asyncio

In [16]:
# First Cell - Backend Setup
import nest_asyncio
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import asyncio
import logging

# Apply nest_asyncio at the start
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Define data model
class ChatRequest(BaseModel):
    message: str

# Define your endpoints
@app.get('/')
async def root():
    return {'welcome': 'to iEraab'}

# Define the FastAPI endpoint
@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    user_message = request.message
    prompt = f"""
انت افضل متخصص فى اعراب النصوص العربية اعرابا مفصلا و دقيقا و طبقا لقواعد اللغة العربية الصحيحة.

القواعد:
- قم بإجراء تحليل نحوي ولغوي مفصل للنص التالي باللغة العربية.
- قم بإعراب النص إعرابا مفصل و دقيق.
- قم بعرض النتائج وفقا للمثال العملى الموضح.

مثال عملى:
"ما أكثرَ الناسَ وما أقلَّ المفيدَ منهم"
ما: اسم تعجب مبني على السكون في محل رفع مبتدأ.
أكثرَ: فعل ماضٍ جامد لإنشاء التعجب مبني على الفتح.
الناسَ: مفعول به منصوب وعلامة نصبه الفتحة الظاهرة على آخره.
و: حرف عطف مبني على الفتح.
ما: اسم تعجب مبني على السكون في محل رفع مبتدأ.
أقلَّ: فعل ماضٍ جامد لإنشاء التعجب مبني على الفتح.
المفيدَ: مفعول به منصوب وعلامة نصبه الفتحة الظاهرة على آخره.
منهم: من حرف جر مبني على السكون، و"هم" ضمير متصل مبني على السكون في محل جر بحرف الجر. والجار والمجرور متعلقان بـ "المفيد"

النص المراد اعرابه:    
{user_message}
    """
    analysis_result = model.generate_text(prompt=prompt)
    return {"response": analysis_result}

# Set up ngrok first
auth_token = "2o1hrQhcITYBZiJMteSIMA0traR_6RHiB2cPxsNXVTPb5xsVX"
ngrok.set_auth_token(auth_token)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)



Public URL: https://ba0b-158-177-149-228.ngrok-free.app


<Task pending name='Task-40' coro=<Server.serve() running at /opt/conda/envs/Python-RT23.1/lib/python3.10/site-packages/uvicorn/server.py:67>>

INFO:     Started server process [267]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [9]:
# Frontend for chat interaction in Jupyter notebook
from IPython.display import display, HTML

html_code = f"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>iEraab - Arabic Grammar Analysis</title>
  <style>
    @import url('https://fonts.googleapis.com/css2?family=Noto+Kufi+Arabic:wght@400;600;700&display=swap');

    body {{
      font-family: 'Noto Kufi Arabic', sans-serif;
      background-color: #f5f5f5;
    }}

    .container {{
      max-width: 800px;
      margin: 0 auto;
      padding: 40px 20px;
    }}

    .header {{
      text-align: center;
      margin-bottom: 30px;
    }}

    .header h1 {{
      color: #0084ff;
      font-size: 36px;
      font-weight: 700;
    }}

    .chat-container {{
      border: 1px solid #ddd;
      padding: 20px;
      height: 450px;
      overflow-y: auto;
      background-color: white;
      border-radius: 10px;
      box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
      margin-bottom: 30px;
    }}

    .user-message, .ai-message {{
      margin: 12px 0;
      padding: 12px 16px;
      border-radius: 20px;
      max-width: 70%;
      font-size: 16px;
      line-height: 1.4;
    }}

    .user-message {{
      background-color: #e6f2ff;
      color: #0077b6;
      align-self: flex-end;
    }}

    .ai-message {{
      background-color: #e6ffe6;
      color: #006400;
      align-self: flex-start;
    }}

    .input-container {{
      display: flex;
      align-items: center;
    }}

    #user-input {{
      flex-grow: 1;
      padding: 12px 16px;
      border: 1px solid #ddd;
      border-radius: 20px;
      font-size: 16px;
    }}

    #send-button {{
      margin-left: 12px;
      padding: 12px 24px;
      background-color: #0084ff;
      color: white;
      border: none;
      border-radius: 20px;
      cursor: pointer;
      font-size: 16px;
      transition: background-color 0.3s;
    }}

    #send-button:hover {{
      background-color: #0069d9;
    }}

    .loading-indicator {{
      display: none;
      margin-left: 12px;
      border: 4px solid #f3f3f3;
      border-top: 4px solid #3498db;
      border-radius: 50%;
      width: 30px;
      height: 30px;
      animation: spin 2s linear infinite;
    }}

    @keyframes spin {{
      0% {{ transform: rotate(0deg); }}
      100% {{ transform: rotate(360deg); }}
    }}
  </style>
</head>
<body>
  <div class="container">
    <div class="header">
      <h1>iEraab - Arabic Grammar Analysis</h1>
      <p>Unlock the secrets of Arabic grammar with our advanced analysis tool.</p>
    </div>
    <div class="chat-container" id="chat-container"></div>
    <div class="input-container">
      <input type="text" id="user-input" placeholder="Enter your Arabic text here..." autocomplete="off">
      <button id="send-button" onclick="sendMessage()">Analyze</button>
      <div class="loading-indicator" id="loading-indicator"></div>
    </div>
  </div>

  <script>
    const API_URL = '{ngrok_tunnel.public_url}/chat';

    async function sendMessage() {{
      const input = document.getElementById('user-input');
      const message = input.value.trim();
      if (message) {{
        const chatContainer = document.getElementById('chat-container');
        const loadingIndicator = document.getElementById('loading-indicator');

        // Display user message
        const userMessage = document.createElement('div');
        userMessage.classList.add('user-message');
        userMessage.textContent = message;
        chatContainer.appendChild(userMessage);

        loadingIndicator.style.display = 'block';
        input.value = '';

        try {{
          const response = await fetch(API_URL, {{
            method: 'POST',
            headers: {{
              'Content-Type': 'application/json'
            }},
            body: JSON.stringify({{ message: message }})
          }});

          if (!response.ok) {{
            throw new Error(`HTTP error! status: ${{response.status}}`);
          }}

          const data = await response.json();
          const analysisResult = data.response || "No response from API.";

          const aiMessage = document.createElement('div');
          aiMessage.classList.add('ai-message');
          aiMessage.textContent = analysisResult;
          chatContainer.appendChild(aiMessage);
        }} catch (error) {{
          const errorMessage = document.createElement('div');
          errorMessage.classList.add('ai-message');
          errorMessage.style.color = 'red';
          errorMessage.textContent = "Error: " + error.message;
          chatContainer.appendChild(errorMessage);
        }} finally {{
          loadingIndicator.style.display = 'none';
        }}

        chatContainer.scrollTop = chatContainer.scrollHeight;
      }}
    }}

    document.getElementById('user-input').addEventListener('keypress', function(event) {{
      if (event.key === 'Enter') {{
        sendMessage();
      }}
    }});
  </script>
</body>
</html>
"""

display(HTML(html_code))

INFO:     85.194.96.74:0 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK
INFO:     85.194.96.74:0 - "POST /chat HTTP/1.1" 200 OK


In [1]:
!pip ngrok

ERROR: unknown command "ngrok"


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [ ]:
prompt_input = """"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [ ]:
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  